In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Conv2D, MaxPooling2D, DepthwiseConv2D, BatchNormalization, LeakyReLU, Concatenate, Input

# 1. Define the Model Architecture (same as the one used during training)
def inceptionv3_base(input_shape=(224, 224, 3)):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3, 3), strides=(2, 2), padding='valid', activation=None)(inputs)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(32, (3, 3), padding='valid', activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(64, (3, 3), padding='same', activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

    x = Conv2D(80, (1, 1), padding='valid', activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(192, (3, 3), padding='valid', activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

    x = GlobalAveragePooling2D()(x)
    return Model(inputs, x)

def custom_cnn_base(input_shape=(224, 224, 3)):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = DepthwiseConv2D((3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(64, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = DepthwiseConv2D((3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(128, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = GlobalAveragePooling2D()(x)
    return Model(inputs, x)

inception_base = inceptionv3_base(input_shape=(224, 224, 3))
custom_cnn_base = custom_cnn_base(input_shape=(224, 224, 3))

inputs = Input(shape=(224, 224, 3))
inception_features = inception_base(inputs)
custom_cnn_features = custom_cnn_base(inputs)

x = Concatenate()([inception_features, custom_cnn_features])
x = Dense(1024, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

hybrid_model = Model(inputs, outputs)

# 2. Load Weights
weights_path = "/content/model_weights.weights.h5"  # Replace with the correct path
hybrid_model.load_weights(weights_path)
print("Model weights loaded successfully!")

# 3. Predict on Input Image
def predict_image(image_path, model, class_names):
    # Load and preprocess the image
    image = load_img(image_path, target_size=(224, 224))  # Resize the image
    image_array = img_to_array(image) / 255.0  # Normalize the image
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

    # Predict
    predictions = model.predict(image_array)
    predicted_class_idx = np.argmax(predictions)
    predicted_class = class_names[predicted_class_idx]
    return predicted_class, predictions[0]

# Define class names (adjust based on your dataset)
class_names = ['Brown_Rust', 'Healthy', 'Yellow_Rust', 'Class4', 'Class5', 'Class6', 'Class7', 'Class8', 'Class9', 'Class10']

# Path to the test image
test_image_path = "/content/yellow_rust_271.png"  # Replace with the actual image path

# Make a prediction
predicted_class, probabilities = predict_image(test_image_path, hybrid_model, class_names)

# Output the result
print(f"Predicted Class: {predicted_class}")
print(f"Class Probabilities: {probabilities}")


Model weights loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
Predicted Class: Yellow_Rust
Class Probabilities: [2.3803649e-02 2.0230493e-06 9.7619438e-01 3.9350665e-09 6.8693793e-09
 5.7335963e-09 3.9057539e-09 3.0889986e-09 2.4611351e-09 3.2281235e-09]
